In [14]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras
import os 
import random 
import cv2
import matplotlib.pyplot as plt

In [15]:
DIRECTORY=r"C:\CSVDviolence,,"
CATAGORIES=["non-violent","violent"]

In [16]:
%%time
img_size=120
seq_len=70

train_data=[]
def create_train_data() :

  for catagory in CATAGORIES :

    path = os.path.join(DIRECTORY,catagory)
    class_num = CATAGORIES.index(catagory) 
    for img in os.listdir(path):

      try:

        img_array=cv2.imread(os.path.join(path,img))
        new_arr=cv2.resize(img_array,(img_size,img_size))
        train_data.append([new_arr,class_num])
      except Exception as e :
        
        pass
create_train_data()

KeyboardInterrupt: 

In [17]:
for sample in train_data[:59400]:
    print(sample[1])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [18]:
%%time
x=[]
y=[]
for features, label in train_data:
    x.append(features)
    y.append(label)
x=np.array(x)
y=np.array(y)

Wall time: 494 ms


In [19]:
%%time
x = x.astype("float32") / 255.0

Wall time: 3.38 s


In [20]:
import pickle 

pickle_out = open('x.pickle',"wb")
pickle.dump(x, pickle_out)
pickle_out.close()
pickle_out = open('y.pickle',"wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
import pickle
xfile = r"F:/saved weights/x.pickle"
X= pickle.load(open(xfile, "rb"))
#y= pickle.load(open(r"F:\saved weights\y.pickle","rb"))

'F:/saved weights/x.pickle'

In [21]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, input_shape = (seq_len, img_size, img_size, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(2, activation = "softmax"))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=True, random_state=0)


In [ ]:
opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

In [ ]:
history = model.fit(x = x_train, y = y_train, epochs=15, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)